In [20]:
import pandas as pd
import numpy as np

In [21]:
data = pd.read_csv('../DataSets/dataus2023.csv')
data.shape

(533088, 21)

In [22]:
firms = pd.read_csv('../DataSets/firmsus2023.csv')
firms.shape

(5615, 7)

Ahora hacemos un merge para adicionar la industri y el status a data:

In [23]:
data['qdate'] = pd.PeriodIndex(data.q, freq="Q")
data


,firm,q,fiscalmonth,revenue,cogs,sgae,otherincome,extraordinaryitems,finexp,incometax,...,inventory,totalliabilities,currentliabilities,longdebt,adjprice,originalprice,sharesoutstanding,fixedassets,year,qdate
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,63.332411,104.0000,452000.000,NaN,2000,2000Q1
1,A,2000q2,6.0,4731000.0,2421000.0,1925000.0,73000.0,0.0,0.0,161000.0,...,1622000.0,2679000.0,2186000.0,0.0,44.911205,73.7500,452271.967,1453000.0,2000,2000Q2
2,A,2000q3,9.0,7401000.0,3790000.0,3016000.0,101000.0,0.0,0.0,244000.0,...,1762000.0,2925000.0,2510000.0,0.0,29.801249,48.9375,453014.579,1581000.0,2000,2000Q3
3,A,2000q4,12.0,10773000.0,5522000.0,4198000.0,111000.0,0.0,0.0,407000.0,...,1853000.0,3160000.0,2758000.0,0.0,33.340861,54.7500,456366.381,1741000.0,2000,2000Q4
4,A,2001q1,3.0,2841000.0,1449000.0,1113000.0,19000.0,-25000.0,0.0,119000.0,...,2129000.0,3667000.0,3070000.0,0.0,18.713509,30.7300,456769.737,1821000.0,2001,2001Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533083,ZY_old,2022q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.780000,2.7800,104098.538,NaN,2022,2022Q3
533084,ZY_old,2022q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.430000,2.4300,104098.538,NaN,2022,2022Q4
533085,ZY_old,2023q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,104098.538,NaN,2023,2023Q1
533086,ZY_old,2023q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,104098.538,NaN,2023,2023Q2


In [24]:
firms.columns
firms1 = firms[["empresa", "Nombre", "status", "naics1"]]
firms1.columns=['firm', 'Empresa', 'status', 'industria']

data =pd.merge(data, firms1, on="firm", how="left")
data


,firm,q,fiscalmonth,revenue,cogs,sgae,otherincome,extraordinaryitems,finexp,incometax,...,longdebt,adjprice,originalprice,sharesoutstanding,fixedassets,year,qdate,Empresa,status,industria
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,63.332411,104.0000,452000.000,NaN,2000,2000Q1,"Agilent Technologies, Inc",activo,Industrias manufactureras
1,A,2000q2,6.0,4731000.0,2421000.0,1925000.0,73000.0,0.0,0.0,161000.0,...,0.0,44.911205,73.7500,452271.967,1453000.0,2000,2000Q2,"Agilent Technologies, Inc",activo,Industrias manufactureras
2,A,2000q3,9.0,7401000.0,3790000.0,3016000.0,101000.0,0.0,0.0,244000.0,...,0.0,29.801249,48.9375,453014.579,1581000.0,2000,2000Q3,"Agilent Technologies, Inc",activo,Industrias manufactureras
3,A,2000q4,12.0,10773000.0,5522000.0,4198000.0,111000.0,0.0,0.0,407000.0,...,0.0,33.340861,54.7500,456366.381,1741000.0,2000,2000Q4,"Agilent Technologies, Inc",activo,Industrias manufactureras
4,A,2001q1,3.0,2841000.0,1449000.0,1113000.0,19000.0,-25000.0,0.0,119000.0,...,0.0,18.713509,30.7300,456769.737,1821000.0,2001,2001Q1,"Agilent Technologies, Inc",activo,Industrias manufactureras
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533083,ZY_old,2022q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.780000,2.7800,104098.538,NaN,2022,2022Q3,Zymergen Inc,cancelado,"Servicios profesionales, científicos y técnicos"
533084,ZY_old,2022q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.430000,2.4300,104098.538,NaN,2022,2022Q4,Zymergen Inc,cancelado,"Servicios profesionales, científicos y técnicos"
533085,ZY_old,2023q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,104098.538,NaN,2023,2023Q1,Zymergen Inc,cancelado,"Servicios profesionales, científicos y técnicos"
533086,ZY_old,2023q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,104098.538,NaN,2023,2023Q2,Zymergen Inc,cancelado,"Servicios profesionales, científicos y técnicos"


In [25]:
data.set_index(["firm", "qdate"], inplace=True)
data

q  fiscalmonth     revenue       cogs       sgae  \
firm   qdate                                                           
A      2000Q1  2000q1          NaN         NaN        NaN        NaN   
       2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
       2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
       2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
       2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   
...               ...          ...         ...        ...        ...   
ZY_old 2022Q3  2022q3          NaN         NaN        NaN        NaN   
       2022Q4  2022q4          NaN         NaN        NaN        NaN   
       2023Q1  2023q1          NaN         NaN        NaN        NaN   
       2023Q2  2023q2          NaN         NaN        NaN        NaN   
       2023Q3  2023q3          NaN         NaN        NaN        NaN   

               otherincome  extraordinaryitems  finexp  incometax  \
firm   qdate                                                        
A      2000Q1          NaN                 NaN     NaN        NaN   
       2000Q2      73000.0                 0.0     0.0   161000.0   
       2000Q3     101000.0                 0.0     0.0   244000.0   
       2000Q4     111000.0                 0.0     0.0   407000.0   
       2001Q1      19000.0            -25000.0     0.0   119000.0   
...                    ...                 ...     ...        ...   
ZY_old 2022Q3          NaN                 NaN     NaN        NaN   
       2022Q4          NaN                 NaN     NaN        NaN   
       2023Q1          NaN                 NaN     NaN        NaN   
       2023Q2          NaN                 NaN     NaN        NaN   
       2023Q3          NaN                 NaN     NaN        NaN   

               totalassets  ...  currentliabilities  longdebt   adjprice  \
firm   qdate                ...                                            
A      2000Q1          NaN  ...                 NaN       NaN  63.332411   
       2000Q2    7321000.0  ...           2186000.0       0.0  44.911205   
       2000Q3    7827000.0  ...           2510000.0       0.0  29.801249   
       2000Q4    8425000.0  ...           2758000.0       0.0  33.340861   
       2001Q1    9208000.0  ...           3070000.0       0.0  18.713509   
...                    ...  ...                 ...       ...        ...   
ZY_old 2022Q3          NaN  ...                 NaN       NaN   2.780000   
       2022Q4          NaN  ...                 NaN       NaN   2.430000   
       2023Q1          NaN  ...                 NaN       NaN        NaN   
       2023Q2          NaN  ...                 NaN       NaN        NaN   
       2023Q3          NaN  ...                 NaN       NaN        NaN   

               originalprice  sharesoutstanding  fixedassets  year  \
firm   qdate                                                         
A      2000Q1       104.0000         452000.000          NaN  2000   
       2000Q2        73.7500         452271.967    1453000.0  2000   
       2000Q3        48.9375         453014.579    1581000.0  2000   
       2000Q4        54.7500         456366.381    1741000.0  2000   
       2001Q1        30.7300         456769.737    1821000.0  2001   
...                      ...                ...          ...   ...   
ZY_old 2022Q3         2.7800         104098.538          NaN  2022   
       2022Q4         2.4300         104098.538          NaN  2022   
       2023Q1            NaN         104098.538          NaN  2023   
       2023Q2            NaN         104098.538          NaN  2023   
       2023Q3            NaN         104098.538          NaN  2023   

                                 Empresa     status  \
firm   qdate                                          
A      2000Q1  Agilent Technologies, Inc     activo   
       2000Q2  Agilent Technologies, Inc     activo   
       2000Q3  Agilent Technologies, Inc     activo   
       200

In [28]:
data['rq'] = np.log(data['adjprice']) - np.log(data.groupby(['firm'])['adjprice'].shift(4))

data['ry'] = np.log(data['adjprice']) - np.log(data.groupby(['firm'])['adjprice'].shift(1))

data.head()


q  fiscalmonth     revenue       cogs       sgae  \
firm qdate                                                           
A    2000Q1  2000q1          NaN         NaN        NaN        NaN   
     2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
     2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
     2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
     2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2      73000.0                 0.0     0.0   161000.0    7321000.0   
     2000Q3     101000.0                 0.0     0.0   244000.0    7827000.0   
     2000Q4     111000.0                 0.0     0.0   407000.0    8425000.0   
     2001Q1      19000.0            -25000.0     0.0   119000.0    9208000.0   

             ...   adjprice  originalprice  sharesoutstanding  fixedassets  \
firm qdate   ...                                                             
A    2000Q1  ...  63.332411       104.0000         452000.000          NaN   
     2000Q2  ...  44.911205        73.7500         452271.967    1453000.0   
     2000Q3  ...  29.801249        48.9375         453014.579    1581000.0   
     2000Q4  ...  33.340861        54.7500         456366.381    1741000.0   
     2001Q1  ...  18.713509        30.7300         456769.737    1821000.0   

             year                    Empresa  status  \
firm qdate                                             
A    2000Q1  2000  Agilent Technologies, Inc  activo   
     2000Q2  2000  Agilent Technologies, Inc  activo   
     2000Q3  2000  Agilent Technologies, Inc  activo   
     2000Q4  2000  Agilent Technologies, Inc  activo   
     2001Q1  2001  Agilent Technologies, Inc  activo   

                             industria        rq        ry  
firm qdate                                                  
A    2000Q1  Industrias manufactureras       NaN       NaN  
     2000Q2  Industrias manufactureras       NaN -0.343710  
     2000Q3  Industrias manufactureras       NaN -0.410137  
     2000Q4  Industrias manufactureras       NaN  0.112233  
     2001Q1  Industrias manufactureras -1.219152 -0.577538  

[5 rows x 25 columns]

Calculate operating profit (also called earnings before interest and taxes) : ebit = revenue - cogs - sgae

In [45]:
data['ebit'] = data['revenue'] - data['cogs'] - data['sgae']

Calculate operating profit margin: opm = ebit / revenue

In [46]:
data['opm'] = data['ebit'] / data['revenue']

Calculate net income as: netincome = ebit + otherincome + extraordinaryitems - financial expenses - incometax

In [51]:
data['netincome'] = data['ebit'] + data['otherincome'] + data['extraordinaryitems'] - data['finexp'] - data['incometax']

Calculate profit margin (ratio) as: pm = ni / revenue

In [47]:
data['pm'] = data['netincome'] / data['revenue']

Calculate asset turn over ratio: ato = revenue / totalassets

In [48]:
data['ato'] = data['revenue'] / data['totalassets']

Calculate acid ratio: acidratio = currentassets / currentliabilities

In [49]:
data['acidratio'] = data['currentassets'] / data['currentliabilities']

Calculate financial leverage ratio as: finlev=longdebt / totalassets

In [50]:
data['finlev'] = data['longdebt'] / data['totalassets']

In [52]:
data.head(10)

q  fiscalmonth     revenue       cogs       sgae  \
firm qdate                                                           
A    2000Q1  2000q1          NaN         NaN        NaN        NaN   
     2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
     2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
     2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
     2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   
     2001Q2  2001q2          6.0   4930000.0  2683000.0  2025000.0   
     2001Q3  2001q3          9.0   6736000.0  3829000.0  3027000.0   
     2001Q4  2001q4         12.0   8396000.0  5166000.0  4008000.0   
     2002Q1  2002q1          3.0   1426000.0   878000.0   990000.0   
     2002Q2  2002q2          6.0   2883000.0  1732000.0  1942000.0   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2      73000.0                 0.0     0.0   161000.0    7321000.0   
     2000Q3     101000.0                 0.0     0.0   244000.0    7827000.0   
     2000Q4     111000.0                 0.0     0.0   407000.0    8425000.0   
     2001Q1      19000.0            -25000.0     0.0   119000.0    9208000.0   
     2001Q2     277000.0            -14000.0     0.0   235000.0    9080000.0   
     2001Q3     312000.0            -19000.0     0.0   142000.0    8622000.0   
     2001Q4     301000.0            580000.0     0.0   -71000.0    7986000.0   
     2002Q1      19000.0              2000.0     0.0  -106000.0    8558000.0   
     2002Q2      41000.0             -4000.0     0.0  -186000.0    8238000.0   

             ...                  industria        rq        ry       ebit  \
firm qdate   ...                                                             
A    2000Q1  ...  Industrias manufactureras       NaN       NaN        NaN   
     2000Q2  ...  Industrias manufactureras       NaN -0.343710   385000.0   
     2000Q3  ...  Industrias manufactureras       NaN -0.410137   595000.0   
     2000Q4  ...  Industrias manufactureras       NaN  0.112233  1053000.0   
     2001Q1  ...  Industrias manufactureras -1.219152 -0.577538   279000.0   
     2001Q2  ...  Industrias manufactureras -0.819441  0.056001   222000.0   
     2001Q3  ...  Industrias manufactureras -0.917569 -0.508265  -120000.0   
     2001Q4  ...  Industrias manufactureras -0.652522  0.377280  -778000.0   
     2002Q1  ...  Industrias manufactureras  0.128965  0.203950  -442000.0   
     2002Q2  ...  Industrias manufactureras -0.317877 -0.390841  -791000.0   

                  opm  netincome        pm       ato  acidratio    finlev  
firm qdate                                                                 
A    2000Q1       NaN        NaN       NaN       NaN        NaN       NaN  
     2000Q2  0.081378   297000.0  0.062777  0.646223   2.313358  0.000000  
     2000Q3  0.080395   452000.0  0.061073  0.945573   2.129084  0.000000  
     2000Q4  0.097744   757000.0  0.070268  1.278694   2.050399  0.000000  
     2001Q1  0.098205   154000.0  0.054206  0.308536   1.778827  0.000000  
     2001Q2  0.045030   250000.0  0.050710  0.542952   1.613299  0.000000  
     2001Q3 -0.017815    31000.0  0.004602  0.781257   1.660485  0.000000  
     2001Q4 -0.092663   174000.0  0.020724  1.051340   2.397103  0.000000  
     2002Q1 -0.309958  -315000.0 -0.220898  0.166628   3.293373  0.134377  
     2002Q2 -0.274367  -568000.0 -0.197017  0.349964   3.269111  0.139597  

[10 rows x 32 columns]